In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [2]:
# Create a dataframe from the results of a BehaviorSpace experiment
def create_xml_file(filename, experiment_name, repetitions, run_metrics_every_step, 
                    setup_command, go_command, time_limit_steps, metrics, run_metrics_condition, enumerated_values):
    # Create the root element
    experiments = ET.Element("experiments")
    
    # Create the experiment element
    experiment = ET.SubElement(experiments, "experiment", 
                               name=experiment_name, 
                               repetitions=str(repetitions), 
                               runMetricsEveryStep=str(run_metrics_every_step).lower())
    ET.indent(experiments)
    
    # Add setup and go commands
    ET.SubElement(experiment, "setup").text = setup_command
    ET.indent(experiment)
    ET.SubElement(experiment, "go").text = go_command
    ET.indent(experiment)
    
    # Add time limit
    ET.SubElement(experiment, "timeLimit", steps=str(time_limit_steps))
    ET.indent(experiment)

    # Add metrics
    for var in metrics:
        ET.SubElement(experiment, "metric").text=var
        ET.indent(experiment)    

    # Add run metrics condition
    ET.SubElement(experiment, "runMetricsCondition").text = run_metrics_condition
    ET.indent(experiment)

    # Add enumerated values
    for var, values in enumerated_values.items():
        evs = ET.SubElement(experiment, "enumeratedValueSet", variable=var)
        for value in values:
            ET.SubElement(evs, "value", value=str(value))
            ET.indent(experiment)    
    # Add stepped values
    #for var, params in stepped_values.items():
    #    ET.SubElement(experiment, "steppedValueSet", variable=var, first=str(params['first']), step=str(params['step']), last=str(params['last']))
    #    ET.indent(experiment)
    
    # Create the tree and write to file
    with open(filename, 'wb') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf8'))
        f.write('<!DOCTYPE experiments SYSTEM "behaviorspace.dtd">\n'.encode('utf8'))
        ET.ElementTree(experiments).write(f, encoding="UTF-8", xml_declaration=False,)

In [3]:
# Example usage
create_xml_file(filename="example.xml", experiment_name="My Experiment", repetitions=10, run_metrics_every_step=False,
            setup_command="setup", 
            go_command="go", 
            time_limit_steps=50000, 
            metrics=["average-whale-hunt","average-help-others","average-fitness"],
            run_metrics_condition='ticks > 30000',
            enumerated_values= {"mutation-prob": [0.1]})

In [4]:
# Split a variable range into a number of splits and sample for LHS
def split_variable_range_and_sample(start,end,splits):
    samples=list()
    for (a,b) in zip(np.linspace(start,end,splits),np.linspace(start,end,splits)[1:]):
        samples.append(np.random.uniform(a,b))
    np.random.shuffle(samples)
    return samples

In [5]:
# Latin Hypercube Sampling
def LHsampling(variables,samples):
    # Create a dataframe with all the combinations of the variables
    df = pd.DataFrame(index=range(samples))
    for var, params in variables.items():
        if params['type'] == 'range':
            df[var] = split_variable_range_and_sample(params['start'],params['end'],samples+1)
        elif params['type'] == 'list':
            df[var] = np.random.choice(params['values'],samples)
    return df

In [6]:
# Netlogo parameters
kawesqar_parameters = {'mu-logistic-whaling-success': {'type': 'range', 'start': 0, 'end': 1.5},
                        'whaling-harm-prob': {'type': 'range', 'start': 0, 'end': 1},
                        'theta': {'type': 'range', 'start': 0, 'end': 1},
                        'mutation-prob': {'type': 'range', 'start': 0, 'end': 0.1},
                        'n-people': {'type': 'list', 'values': [1000]}}

In [7]:
# ALtamira grid workin directory
path="C:/Users/jisma/altamira/kawesqar24"
#path="C:/Users/nacho/altamira/kawesqar24"


In [35]:
# Generate LH sampling
# Run only once !!!!!!!!!!!!!!!!!!!!
NLHS=1000
#lhs=LHsampling(kawesqar_parameters,NLHS)
#lhs.to_csv(path+'/experiments/lhs.csv')
#lhs.head()

,mu-logistic-whaling-success,whaling-harm-prob,theta,mutation-prob,n-people
0,0.970502,0.730955,0.729608,0.056979,1000
1,1.205201,0.750981,0.653467,0.017585,1000
2,1.274088,0.507802,0.634778,0.048723,1000
3,1.021986,0.871361,0.711575,0.043537,1000
4,0.513559,0.985016,0.845251,0.019079,1000


In [8]:
# Read LH sampling
lhs=pd.read_csv(path+'/experiments/lhs.csv',index_col=0)
lhs.head()

,mu-logistic-whaling-success,whaling-harm-prob,theta,mutation-prob,n-people
0,0.970502,0.730955,0.729608,0.056979,1000
1,1.205201,0.750981,0.653467,0.017585,1000
2,1.274088,0.507802,0.634778,0.048723,1000
3,1.021986,0.871361,0.711575,0.043537,1000
4,0.513559,0.985016,0.845251,0.019079,1000


In [9]:
# Create the XML files for the experiments
for i in lhs.index:
    create_xml_file(filename=path+'/experiments/experiment_'+str(i)+'.xml', 
            experiment_name="lhs", repetitions=10, run_metrics_every_step=False,
            setup_command="startup", 
            go_command="go", 
            time_limit_steps=50000, 
            metrics=["average-whale-hunt","average-help-others","average-fitness","average-resources",
                     "average-social-capital","average-disabled"],
            run_metrics_condition='ticks > 30000',
            enumerated_values= {k:[lhs.loc[i,k]] for k in lhs.columns})

In [10]:
# Altamira paths
netlogo_path='/gpfs/res_projects/uc27/netlogo6.4.0/NetLogo_Console'
model_path='/home/uc27/uc27001/kawesqar24/model/kawesqar_v01.nlogo'
experiment_path='/home/uc27/uc27001/kawesqar24/experiments'
results_path='/home/uc27/uc27001/kawesqar24/results'

# Local path
#batch_path='C:/Users/jisma/altamira/kawesqar24/batches'
batch_path='C:/Users/nacho/altamira/kawesqar24/batches_c'


In [11]:
# Create shell script to run the experiments
for i in lhs.index:
    with open(batch_path + '/run_experiment_' + str(i) +'.sh','w') as f:
        f.write("#!/bin/bash\n")
        f.write(netlogo_path + ' --headless --model ' + model_path + 
                            ' --setup-file ' + experiment_path + '/experiment_' + str(i) + '.xml ' +  '--experiment "lhs"' +
                            ' --table ' + results_path + '/result_table_exp_' + str(i) + '.csv' +
                            ' --stats ' + results_path + '/result_stats_exp_' + str(i) + '.csv' +
                            ' --threads 16\n')
    f.close()

In [ ]:
# 50 jobs of 20 tasks (=nodes)
# Nb=50

Create 100 jobs of 10 tasks (=Altamira nodes)

In [12]:
# 100 jobs of 10 tasks (=nodes)
Nb=100 # Number of batches

In [15]:
# Create the slurm shell scripts for each bacth (job) 
Ne=lhs.shape[0] # Number of experiments

M=int(np.ceil(Ne/Nb)) # Number of experiments per batch
for j in range(Nb):
    i1=j*M
    i2=((j+1)*M) if ((j+1)*M)<=Ne else Ne
    with open(batch_path+'/batch_'+str(j) + '.sh','w') as f:
        f.write("#!/bin/bash\n")
        f.write("#SBATCH --job-name=batch_"+str(j)+"\n")
        f.write("#SBATCH --output=batch_"+str(j)+".out\n")
        f.write("#SBATCH --error=batch_"+str(j)+".err\n")
        f.write("#SBATCH --time=3:00:00\n")
        f.write("#SBATCH --ntasks="+str(i2-i1)+"\n") # M tasks per job
        f.write("#SBATCH --cpus-per-task=16\n")
        f.write('export JAVA_OPTS="-Xmx3072m -XX:CompressedClassSpaceSize=1024m"\n')
        f.write("start="+str(i1)+"\n")
        f.write("end="+str(i2)+"\n")
        f.write("for ((i=start; i<end; i++)); do\n")
        f.write('\tfilename="run_experiment_$i.sh"\n')
        f.write("\tsrun --exclusive --ntasks=1 --cpus-per-task=$SLURM_CPUS_PER_TASK ./$filename &\n")  
        f.write("done\n")
        f.write("wait\n")
    f.close()
